In [ ]:
def chunks(arr, n_arr):
    
    n_elems = int(math.ceil(len(arr)/n_arr))
    return [arr[i:i + n_elems] for i in range(0, len(arr), n_elems)]

print('\tchunks() loaded')

In [ ]:
def part_histograms(img, x_parts=4, y_parts=3, bins=1):
    
    (x_dim, y_dim) = img.shape
    x_pix, y_pix = chunks(range(x_dim),x_parts), chunks(range(y_dim),y_parts)
    
    part_array = np.array([])
    
    for j in range(len(x_pix)):
        x_range = x_pix[j]

        for k in range(len(y_pix)):
            y_range = y_pix[k]

            indeces = np.ix_(x_range, y_range)
            img_part = img[indeces]
            if bins == 1:
                hist = img_part.sum()
            else:
                hist, bin_edges = np.histogram(img_part.ravel(), range=(1e-5,np.max(img.ravel())), bins=bins)

            part_array = np.hstack((part_array, hist))
        
    return part_array

print('\tpart_histograms() loaded')

In [ ]:
def cropImage(img, img_segmentation, crop=True):
    
    if crop == True:
        mask = np.mean(img_segmentation, axis=2) > 150 # For depth images
        if img.ndim == 3:
            mask = np.dstack((mask,mask,mask))

        x_zero = np.where(np.all(mask==0, axis=1) == 1)
        y_zero = np.where(np.all(mask==0, axis=0) == 1)

        img = np.delete(img, x_zero, axis=0)
        img = np.delete(img, y_zero, axis=1)

        mask = np.delete(mask, x_zero, axis=0)
        mask = np.delete(mask, y_zero, axis=1)
    else:
        mask = np.mean(img_segmentation, axis=2) > 150 # For depth images
        
    return img, mask

print("\tcropImage() loaded")

In [ ]:
def rescaleDepthImage(img, img_segmentation, std=3):
    
    _, mask = cropImage(img, img_segmentation, False)
    ma = img[np.nonzero(img*mask)]
    img_min = max(np.mean(ma) - std * np.std(ma), np.min(ma))
    img_max = min(np.mean(ma) + std * np.std(ma), np.max(ma))
    
    img = (img - img_min)/(img_max - img_min)*255*mask
    img = cv2.convertScaleAbs(img)
    
    return img

print('\trescaleDepthImage loaded')

In [ ]:
def train(X_train, labels_train, method='rf', use_pca=False):
    
    if use_pca == True:
        param_svm ={
        "pca__n_components": [20, 40, 60],
        "svm__kernel": ['rbf'],
        "svm__gamma": [1e-3, 1e-4],
        "svm__C": [0.01, 1, 10, 100]}
        pipe_svm = Pipeline([('pca', PCA()), ('svm', SVC())])
        clf_svm = GridSearchCV(pipe_svm, param_svm, cv=5, n_jobs=8)
    else:
        param_svm ={
        "kernel": ['rbf'],
        "gamma": [1e-3, 1e-4],
        "C": [0.01, 1, 10, 100]}
        clf_svm = GridSearchCV(SVC(), param_svm, cv=5, n_jobs=4)
    
    param_rf = {
        "max_depth": [None],
        "max_features": [20, 40],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 3, 10],
        "n_estimators": [6, 10, 20],
        "bootstrap": [True, False],
        "criterion": ["entropy", "gini"]}
    clf_rf = GridSearchCV(RandomForestClassifier(n_jobs=4), param_rf, cv=5, n_jobs=4)
    
    if use_pca == True:
        param_lr = {
            "pca__n_components": [20, 40, 60],
            "logistic__C" : np.logspace(-4, 4, 3)}
        pipe_lr = Pipeline([('pca', PCA()), ('logistic', LogisticRegression(n_jobs=4))])
        clf_lr = GridSearchCV(pipe_lr, param_lr, cv=5, n_jobs=4)
    else:
        param_lr = {
            "C" : np.logspace(-4, 4, 3)}
        clf_lr = GridSearchCV(LogisticRegression(n_jobs=4), param_lr, cv=5, n_jobs=4)
    
    if method == 'svm':
        clf = clf_svm
    elif method == 'rf':
        clf = clf_rf
    elif method == 'lr':
        clf = clf_lr
    elif method == 'voting':
        clf = VotingClassifier([('lr', clf_lr), ('rf', clf_rf), ('svm', clf_svm)], voting='hard')
    
    clf.fit(X_train, labels_train)

    return clf

print('\ttrain() loaded')